# Brandenburg

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Brandenburg is `DE4`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [13]:
import pandas as pd
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict, Tuple
import zipfile
from datetime import datetime as dt
from io import StringIO
import warnings
from dateparser import parse

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Brandenburg').input_path
BASE

'/home/alexander/Github/camels/camelsp/input_data/BR_Brandenburg'

### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

Nice, Brandenburg is quite cool. This will be hard to parse. Let's first extract the big Zip, because it's a zip of Excel, with MANY sheets. To make thing more complicated, they splitted the Excel files into two files, I guess because they got too large (haha).

In [3]:
# extract the ZIP in place
if not os.path.exists(os.path.join(BASE, 'Q.TagWerte_1.xlsx')):
    with zipfile.ZipFile(os.path.join(BASE, 'Anlage_4_W_Q-TagWerte.zip')) as z:
        for f in z.filelist:
            z.extract(f, BASE)

Extract the metadata from each sheet, for this, first read in all files and create a mapping from station ids to the sheet in the respective excel_file. 

Ah, nice, why call them by their name if you can just call them 'Zeitreihe_1', 'Zeitreihe_2' and so on?

In [4]:
q1 = pd.ExcelFile(os.path.join(BASE, 'Q.TagWerte_1.xlsx'))
q2 = pd.ExcelFile(os.path.join(BASE, 'Q.TagWerte_2.xlsx'))
w1 = pd.ExcelFile(os.path.join(BASE, 'W.TagWerte_1.xlsx'))
w2 = pd.ExcelFile(os.path.join(BASE, 'W.TagWerte_2.xlsx'))

In [5]:
name = 'Zeitreihe_1'
df = q1.parse(name, header=None, usecols=[0,1,2,3])
df

,0,1,2,3
0,Name,"Alt Ruppin, Schleuse OP.Q.TagWerte",NaN,NaN
1,ID,165666,NaN,NaN
2,Parameter,Durchfluss,NaN,NaN
3,Einheit,m³/s,NaN,NaN
4,Anzahl Messwerte,15327,NaN,NaN
...,...,...,...,...
15348,2022-10-14 00:00:00,1.2,131072,Ungeprüft
15349,2022-10-15 00:00:00,1.22,131072,Ungeprüft
15350,2022-10-16 00:00:00,1.21,131072,Ungeprüft
15351,2022-10-17 00:00:00,1.2,131072,Ungeprüft


This will be a tough sunday.

The header is changing its size. I hope the blocks stay the same...

In [6]:
def parse_dirty_dataframe(df: pd.DataFrame, variable: str, skip_data: bool = False) -> Tuple[Dict, pd.DataFrame]:
    """
    Parse the dirty dataframe directly read from the excel sheets.

    """
    # get all empty row, as they separate the block, of course
    null_idx = df[df.isnull().all(axis=1) == True].index.to_list()

    # mark all block as None for now
    ID = None
    base = {}
    loc = {}
    co = {}
    dat = None

    # We expect three blocks here, base metadata, location metadata, coordinates and data
    for i, (lo, up) in enumerate(zip([0] + null_idx, null_idx + [len(df)])):
        # extract the block
        block = df.iloc[lo:up, :]
        
        # switch the block
        # data block - skipping for now
        if i == 3 and not skip_data:
            # on i==0, the ID should be filled, otherwise we have to handcraft this data
            if ID is None:
                continue
            
            # get the block
            block = block.dropna(axis=0, how='all').dropna(axis=1, how='all')

            # set the header
            block.columns = block.iloc[0, :]
            block.drop(block.index[0], axis=0, inplace=True)

            # transform the data as everything is string, of course
            dat = pd.DataFrame({
                'date': [d.date() if isinstance(d, dt) else parse(d) for d in block.iloc[:, 0].values],
                variable.lower(): block.iloc[:, 1].astype(float),
                'flag': [fl.strip().lower() == 'geprüft' for fl in block.iloc[:, 3].values]
            })
        
        # base data or Standord
        if i == 0 or i == 1:
            block = block.iloc[:, :2].dropna(axis=1, how='all').dropna(axis=0, how='all')
            block = block.set_index(0).T.to_dict(orient='records')
            if len(block) > 0:
                if i == 0:
                    if 'ID' in block[0].keys():
                        ID = str(block[0]['ID'])
                    else:
                        warnings.warn(f"Block #{i + 1}: No ID found. This will skip the data for this station.")
                    base = block[0]
                else:
                    loc = block[0]
            else:
                warnings.warn(f"Block #{i + 1}: did not yield the correct shape. Please check the file. Skipping.")
        
        # Koordinaten
        elif i == 2:
            # drop NaNs on both axis
            block = block.dropna(axis=1, how='all').dropna(axis=0, how='all')
            
            # rename the CRS header
            block.iloc[0, 0] = 'CRS'
            block.columns = block.iloc[0]
            block.drop(block.index[0], axis=0, inplace=True)
            block = block.to_dict(orient='records')

            if len(block) > 0:
                co = block[0]
            else:
                warnings.warn(f"Block #{i + 1} did not yield the correct shape. Please check the file. Skipping.")

    # now merge the metadata
    meta = {**base, **loc, **co}

    # if there was no metadata, set meta None again
    if len(meta.keys()) == 0:
        meta = None
    
    # finally return all we got
    return meta, dat


# Test the stuff
parse_dirty_dataframe(df, 'q', skip_data=False)


({'Name': 'Alt Ruppin, Schleuse OP.Q.TagWerte',
  'ID': '165666',
  'Parameter': 'Durchfluss',
  'Einheit': 'm³/s',
  'Anzahl Messwerte': '15327',
  'Messwerte von': '01.11.1980 00:00:00',
  'Messwerte bis': '18.10.2022 00:00:00',
  'Name vom Messort/ Messgebiet': 'Alt Ruppin, Schleuse OP, 5891200',
  'Pegelname': 'Alt Ruppin, Schleuse OP',
  'Gewaesser': 'Rhin (Ruppiner Wasserstraße)',
  'Status': 'Aktuell',
  'Betreiber': 'LfU Brandenburg',
  'PNP_Höhenstatus': 'DHHN2016',
  'PNP_Höhe': '34.965',
  'Fluss_KM': '49.901',
  'FlussohMdg': '45.42',
  'Gew_Kennz': '588',
  'Meldewesen': 'täglicher Meldedienst',
  'Pegelklasse': 'Gewässerkundliches Netz (GK)',
  'Messstellennummer': '5891200',
  'CRS': 'UTM 33N',
  'Ost/RW': '354590.00',
  'Nord/HW': '5869955.00'},
              date     q   flag
 26     1980-11-01  2.20   True
 27     1980-11-02  2.20   True
 28     1980-11-03  2.20   True
 29     1980-11-04  2.20   True
 30     1980-11-05  2.20   True
 ...           ...   ...    ...
 153

## Generate metadata and data in Memory

I hope all the stuff can be put into Memory at once, otherwise we have to chunk it.

In [7]:
# empty container for the data
metadata, data, warns = [], [], []

with warnings.catch_warnings(record=True) as wa:
    # extract from each of the four excel sheets
    for variable, xls in zip(('q', 'q', 'w', 'w'), (q1, q2, w1, w2)):
        # go for each sheet
        for sheet_name in tqdm(xls.sheet_names):
            # load the dirty sheet
            df = xls.parse(sheet_name, header=None, usecols=[0,1,2,3])

            # parse it
            meta, dat = parse_dirty_dataframe(df, variable)

            if meta is not None:
                metadata.append(meta),
                data.append(dat)

    # copy warnings
    warns.extend(wa)

print(f"metadata length: {len(metadata)}    data files: {len(data)}      warnings:{len(warns)}")

100%|██████████| 115/115 [01:23<00:00,  1.38it/s]

metadata length: 382    data files: 382      warnings:0


In [21]:
dat

,date,w,w_flag
29,1977-11-01,36.0,True
30,1977-11-02,36.0,True
31,1977-11-03,42.0,True
32,1977-11-04,42.0,True
33,1977-11-05,42.0,True
...,...,...,...
16444,2022-10-13,75.0,False
16445,2022-10-14,75.0,False
16446,2022-10-15,76.0,False
16447,2022-10-16,75.0,False


## There is more

We have not only Anlage 4, but also Anlage 3. Check this file out:

In [8]:
# define the function 
def read_meta(base_path) -> pd.DataFrame:
    path = os.path.join(base_path, 'Anlage_3.xlsx')
    meta = pd.read_excel(path)
    return meta

# test it here
other_meta = read_meta(BASE)
pmeta = pd.DataFrame(metadata)

# merge with the other, more interesting metadata
#meta = pd.merge(pd.DataFrame(metadata), other_meta, left_on='ID', right_on='station_no', how='inner')

print(f"Overlapping names: {any([n in pmeta.Name.values for n in other_meta.station_name])}")
print(f"Overlapping Messstellennummer: {any([str(i) in pmeta.Messstellennummer.values.tolist() for i in other_meta.station_no])}")

Overlapping names: False
Overlapping Messstellennummer: True


/home/alexander/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
# join both metadata together
all_meta = pmeta.join(other_meta.set_index('station_no'), on='Messstellennummer', how='left')
all_meta

,Name,ID,Parameter,Einheit,Anzahl Messwerte,Messwerte von,Messwerte bis,Name vom Messort/ Messgebiet,Pegelname,Gewaesser,...,Messstellennummer,CRS,Ost/RW,Nord/HW,W_seit,W_bis,Q_seit,station_name,CATCHMENT_SIZE,BODY_RESPONSIBLE
0,"Alt Ruppin, Schleuse OP.Q.TagWerte",165666,Durchfluss,m³/s,15327,01.11.1980 00:00:00,18.10.2022 00:00:00,"Alt Ruppin, Schleuse OP, 5891200","Alt Ruppin, Schleuse OP",Rhin (Ruppiner Wasserstraße),...,5891200,UTM 33N,354590.00,5869955.00,NaN,NaN,NaN,"Alt Ruppin, Schleuse OP","533,24 km²",LfU Brandenburg
1,"Altlandsberg 2, Walkmühle.Q.TagWerte",278150,Durchfluss,m³/s,11990,01.11.1987 00:00:00,31.08.2022 00:00:00,"Altlandsberg 2, Walkmühle, 5861002","Altlandsberg 2, Walkmühle",Erpe/Neuenhagener Fließ,...,5861002,UTM 33N,413345.00,5825130.00,24.08.1987,aktuell,01.11.1987,"Altlandsberg 2, Walkmühle","118,21 km²",LfU Brandenburg
2,Babelsberg-Drewitz.Q.TagWerte,166379,Durchfluss,m³/s,24824,01.11.1954 00:00:00,18.10.2022 00:00:00,"Babelsberg-Drewitz, 5871600",Babelsberg-Drewitz,Nuthe,...,5871600,UTM 33N,371737.00,5803058.00,NaN,NaN,NaN,Babelsberg-Drewitz,"1792,07 km²",LfU Brandenburg
3,Bad Liebenwerda.Q.TagWerte,166369,Durchfluss,m³/s,23364,01.11.1960 00:00:00,19.10.2022 00:00:00,"Bad Liebenwerda, 5530500",Bad Liebenwerda,Schwarze Elster,...,5530500,UTM 33N,388602.00,5708769.00,01.12.1887,aktuell,01.11.1920,Bad Liebenwerda,"3154,03 km²",LfU Brandenburg
4,Bad Wilsnack.Q.TagWerte,165980,Durchfluss,m³/s,16056,01.11.1975 00:00:00,16.10.2022 00:00:00,"Bad Wilsnack, 5930500",Bad Wilsnack,Karthane,...,5930500,UTM 33N,294280.00,5871815.00,NaN,NaN,NaN,Bad Wilsnack,"284,62 km²",LfU Brandenburg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,Wünsdorf 1 UP.W.TagWerte,301037,Wasserstand,cm,14184,01.11.1983 00:00:00,31.08.2022 00:00:00,"Wünsdorf 1 UP, 5865501",Wünsdorf 1 UP,Wünsdorfer Kanal,...,5865501,UTM 33N,393811.00,5778777.00,NaN,17.05.1983,17.05.1983,Wünsdorf 1 UP,"47,31 km²",LfU Brandenburg
378,Wünsdorf 2 OP.W.TagWerte,301044,Wasserstand,cm,19191,01.11.1969 00:00:00,31.08.2022 00:00:00,"Wünsdorf 2 OP, 5865600",Wünsdorf 2 OP,Neuer Graben,...,5865600,UTM 33N,394693.00,5779961.00,01.08.1969,aktuell,01.08.1969,Wünsdorf 2 OP,"0,04 km²",LfU Brandenburg
379,Wünsdorf.W.TagWerte,301055,Wasserstand,cm,14183,01.11.1983 00:00:00,31.08.2022 00:00:00,"Wünsdorf, 5865502",Wünsdorf,Luchgraben,...,5865502,UTM 33N,393821.00,5778750.00,17.05.1983,aktuell,17.05.1983,Wünsdorf,"47,31 km²",LfU Brandenburg
380,"Zeuthen, Straßenbrücke.W.TagWerte",301061,Wasserstand,cm,17123,01.11.1975 00:00:00,17.10.2022 00:00:00,"Zeuthen, Straßenbrücke, 5865900","Zeuthen, Straßenbrücke",Selchower Flutgraben,...,5865900,UTM 33N,406492.00,5801049.00,09.06.1975,aktuell,01.11.1975,"Zeuthen, Straßenbrücke","76,01 km²",LfU Brandenburg


In [10]:
# the id column will be Messstellennummer
id_column = 'ID'

Check that we have an ID everywhere

In [11]:
print(f"All stations have an id: {all(['ID' in m for m in metadata])}")
print(f"Any duplicated IDs:      {pd.DataFrame(metadata).ID.duplicated().any()}")

All stations have an id: True
Any duplicated IDs:      False


### Finally run

Save the stuff using the context

In [12]:
with Bundesland('Brandenburg') as bl:
    # save the metadata
    bl.save_raw_metadata(all_meta, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())
    
    # go for each of the files
    for station_meta, station_data in tqdm(zip(metadata, data), total=len(metadata)):
        # get the provider id
        provider_id = station_meta[id_column]
        
        # save
        bl.save_timeseries(station_data, provider_id)

    # check if there were warnings (there are warnings)
    if len(warns) > 0:
        log_path = bl.save_warnings(warns)
        print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DE410000      165666  ./DE4/DE410000/DE410000_data.csv
1  DE410010      278150  ./DE4/DE410010/DE410010_data.csv
2  DE410020      166379  ./DE4/DE410020/DE410020_data.csv
3  DE410030      166369  ./DE4/DE410030/DE410030_data.csv
4  DE410040      165980  ./DE4/DE410040/DE410040_data.csv


  0%|          | 0/382 [00:00<?, ?it/s]

100%|██████████| 382/382 [00:20<00:00, 18.28it/s]


In [16]:
station_data

,date,w,w_flag
29,1977-11-01,36.0,True
30,1977-11-02,36.0,True
31,1977-11-03,42.0,True
32,1977-11-04,42.0,True
33,1977-11-05,42.0,True
...,...,...,...
16444,2022-10-13,75.0,False
16445,2022-10-14,75.0,False
16446,2022-10-15,76.0,False
16447,2022-10-16,75.0,False
